In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from utils.data import load_tc_data
from utils.plot import make_cmap
import cftime as cf



In [2]:
load_from_scratch=True
plt.rcParams.update({
    # Use LaTeX default serif font-family.
    # https://matplotlib.org/stable/tutorials/text/usetex.html#sphx-glr-tutorials-text-usetex-py
     "font.family": "sans-serif",
     # Use specific cursive fonts.
     "font.cursive": ["Comic Neue", "Comic Sans MS"],
 })
font = {'fontname':'Comic Neue'}

In [28]:
tracks = pd.read_csv('/user/home/al18709/work/ibtracks/ibtracs.ALL.list.v04r00.csv')

/tmp/ipykernel_130596/901849473.py:1: DtypeWarning: Columns (1,2,8,9,14,19,20,25,161,162) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv('/user/home/al18709/work/ibtracks/ibtracs.ALL.list.v04r00.csv')


In [33]:
print(tracks)

                  SID SEASON NUMBER BASIN SUBBASIN       NAME  \
0                       Year                                    
1       1842298N11080   1842      1    NI       BB  NOT_NAMED   
2       1842298N11080   1842      1    NI       BB  NOT_NAMED   
3       1842298N11080   1842      1    NI       BB  NOT_NAMED   
4       1842298N11080   1842      1    NI       BB  NOT_NAMED   
...               ...    ...    ...   ...      ...        ...   
699909  2021236N12296   2021     65   NaN       GM        IDA   
699910  2021236N12296   2021     65   NaN       GM        IDA   
699911  2021236N12296   2021     65   NaN       GM        IDA   
699912  2021236N12296   2021     65   NaN       GM        IDA   
699913  2021236N12296   2021     65   NaN       GM        IDA   

                   ISO_TIME NATURE            LAT           LON  ...  \
0                                   degrees_north  degrees_east  ...   
1       1842-10-25 03:00:00     NR        10.9000       80.3000  ...   
2  

In [15]:
def find_dates(sids,model_cal):

	dates = sids.apply(lambda row : cf.datetime(calendar=model_cal,
									year=row.year,
									month=row.month,
									day=row.day,
									hour=row.hour
									), axis=1)
	return dates



In [34]:
def lookup(row):
	time = tracks.loc[(tracks.SID == row.sid) and(tracks.LAT == row.lat) and (tracks.LON == row.lon)]
	
	date = cf.datetime(calendar='gregorian',
					year=time.ISOTIME.dt.year,
					month=time.ISOTIME.dt.month,
					day=time.ISOTIME.dt.day,
					hour=time.ISOTIME.dt.hour
					)
	return date



def find_dates_mswep(meta):

	# find matching sid
	# find matching lat lon
	# meta[i].ISOTIME.dt.year
	dates = meta.apply(lookup, axis=1)

	return dates

In [35]:
print(tracks)

                  SID SEASON NUMBER BASIN SUBBASIN       NAME  \
0                       Year                                    
1       1842298N11080   1842      1    NI       BB  NOT_NAMED   
2       1842298N11080   1842      1    NI       BB  NOT_NAMED   
3       1842298N11080   1842      1    NI       BB  NOT_NAMED   
4       1842298N11080   1842      1    NI       BB  NOT_NAMED   
...               ...    ...    ...   ...      ...        ...   
699909  2021236N12296   2021     65   NaN       GM        IDA   
699910  2021236N12296   2021     65   NaN       GM        IDA   
699911  2021236N12296   2021     65   NaN       GM        IDA   
699912  2021236N12296   2021     65   NaN       GM        IDA   
699913  2021236N12296   2021     65   NaN       GM        IDA   

                   ISO_TIME NATURE            LAT           LON  ...  \
0                                   degrees_north  degrees_east  ...   
1       1842-10-25 03:00:00     NR        10.9000       80.3000  ...   
2  

In [36]:
# load current 1D dataset
real,inputs,pred,meta = load_tc_data(set='validation',results='ke_tracks')
meta['date'] = find_dates_mswep(meta)

# load ibtracks
tracks = pd.read_csv('/user/home/al18709/work/ibtracks/tc_files.csv')

og_tracks_grouped = tracks.groupby('sid')


# load original 2D WGAN
real_2,inputs_2,pred_2,meta_2,imput_og,pred_og,meta_og = load_tc_data(set='validation',results='kh_tracks')
meta_2['date'] = find_dates(meta_2,'gregorian')
meta_og['date'] = find_dates(meta_og,'gregorian')
real_og_x,_,_,_,_,_,pred_og_x,meta_og_x = load_tc_data(set='extreme_test',results='test')
meta_og_x['date'] = find_dates(meta_og_x,'gregorian')
meta_og = pd.read_csv('/user/work/al18709/tc_data_mswep_40/valid_meta.csv')
# meta_valid = pd.read_csv('/user/work/al18709/tc_data_flipped/valid_meta.csv')

# load imerg
imerg_rain = np.load('/user/home/al18709/work/CMIP6/IMERG-V07/storm_rain/storm_rain_NH.npy')
imerg_lats = np.load('/user/home/al18709/work/CMIP6/IMERG-V07/storm_rain/storm_lats_NH.npy')
imerg_lons = np.load('/user/home/al18709/work/CMIP6/IMERG-V07/storm_rain/storm_lons_NH.npy')
imerg_sids = pd.read_csv('/user/home/al18709/work/CMIP6/IMERG-V07/storm_rain/storm_sid_NH.csv')
print(imerg_sids.sid.shape)
meta_imerg = pd.DataFrame({'sid':imerg_sids.sid ,'centre_lat':imerg_lats[:,4], 'centre_lon':imerg_lons[:,4],'date':find_dates(imerg_sids,'julian')})


# load HighResMIP CMCC-CM2-VHR4 historical NH
CMCC_hist_NH_rain = np.load('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_rain_NH.npy')
CMCC_hist_NH_lats = np.load('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_lats_NH.npy')
CMCC_hist_NH_lons = np.load('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_lons_NH.npy')
CMCC_hist_NH_sids = pd.read_csv('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_sid_NH.csv')
print(CMCC_hist_NH_sids.sid.shape)
meta_CMCC_hist_NH = pd.DataFrame({'sid':CMCC_hist_NH_sids.sid ,'centre_lat':CMCC_hist_NH_lats[:,4], 'centre_lon':CMCC_hist_NH_lons[:,4], 'date':find_dates(CMCC_hist_NH_sids,'365_day')})

# load HighResMIP CMCC-CM2-VHR4 historical SH
CMCC_hist_SH_rain = np.load('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_rain_SH.npy')
CMCC_hist_SH_lats = np.load('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_lats_SH.npy')
CMCC_hist_SH_lons = np.load('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_lons_SH.npy')
CMCC_hist_SH_sids = pd.read_csv('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_sid_SH.csv')
print(CMCC_hist_SH_sids.sid.shape)
meta_CMCC_hist_SH = pd.DataFrame({'sid':CMCC_hist_SH_sids.sid ,'centre_lat':CMCC_hist_SH_lats[:,4], 'centre_lon':CMCC_hist_SH_lons[:,4], 'date':find_dates(CMCC_hist_SH_sids,'365_day')})

# load HighResMIP EC-EARTH-3P-HR  historical NH
ecearthhr_hist_NH_rain = np.load('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_rain_NH.npy')
ecearthhr_hist_NH_lats = np.load('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_lats_NH.npy')
ecearthhr_hist_NH_lons = np.load('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_lons_NH.npy')
ecearthhr_hist_NH_sids = pd.read_csv('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_sid_NH.csv')
print(ecearthhr_hist_NH_sids.sid.shape)
meta_ecearthhr_hist_NH = pd.DataFrame({'sid':ecearthhr_hist_NH_sids.sid ,'centre_lat':ecearthhr_hist_NH_lats[:,4], 'centre_lon':ecearthhr_hist_NH_lons[:,4], 'date':find_dates(ecearthhr_hist_NH_sids,'proleptic_gregorian')})

# load HighResMIP EC-EARTH-3P-HR historical SH
ecearthhr_hist_SH_rain = np.load('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_rain_SH.npy')
ecearthhr_hist_SH_lats = np.load('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_lats_SH.npy')
ecearthhr_hist_SH_lons = np.load('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_lons_SH.npy')
ecearthhr_hist_SH_sids = pd.read_csv('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_sid_SH.csv')
print(ecearthhr_hist_SH_sids.sid.shape)
meta_ecearthhr_hist_SH = pd.DataFrame({'sid':ecearthhr_hist_SH_sids.sid ,'centre_lat':ecearthhr_hist_SH_lats[:,4], 'centre_lon':ecearthhr_hist_SH_lons[:,4], 'date':find_dates(ecearthhr_hist_SH_sids,'proleptic_gregorian')})

# load CMIP6 historical dataset 
miroc6_hist_rain = np.load('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_rain_NH.npy')
miroc6_hist_lats = np.load('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_lats_NH.npy')
miroc6_hist_lons = np.load('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_lons_NH.npy')
miroc6_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_sid_NH.csv')
meta_miroc6_hist = pd.DataFrame({'sid':miroc6_hist_sids.sid ,'centre_lat':miroc6_hist_lats[:,4], 'centre_lon':miroc6_hist_lons[:,4], 'date':find_dates(miroc6_hist_sids,'gregorian')})

miroc6_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_rain_SH.npy')
miroc6_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_lats_SH.npy')
miroc6_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_lons_SH.npy')
miroc6_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_sid_SH.csv')
meta_miroc6_hist_SH = pd.DataFrame({'sid':miroc6_hist_sids_SH.sid ,'centre_lat':miroc6_hist_lats_SH[:,4], 'centre_lon':miroc6_hist_lons_SH[:,4], 'date':find_dates(miroc6_hist_sids_SH,'gregorian')})

# CMIP6 MPI-ESM1-2-LR
mpi_hist_rain = np.load('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_rain_NH.npy')
mpi_hist_lats = np.load('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_lats_NH.npy')
mpi_hist_lons = np.load('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_lons_NH.npy')
mpi_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_sid_NH.csv')
meta_mpi_hist = pd.DataFrame({'sid':mpi_hist_sids.sid ,'centre_lat':mpi_hist_lats[:,4], 'centre_lon':mpi_hist_lons[:,4], 'date':find_dates(mpi_hist_sids,'proleptic_gregorian')})

mpi_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_rain_SH.npy')
mpi_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_lats_SH.npy')
mpi_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_lons_SH.npy')
mpi_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_sid_SH.csv')
meta_mpi_hist_SH = pd.DataFrame({'sid':mpi_hist_sids_SH.sid ,'centre_lat':mpi_hist_lats_SH[:,4], 'centre_lon':mpi_hist_lons_SH[:,4], 'date':find_dates(mpi_hist_sids_SH,'proleptic_gregorian')})

# CMIP6 Noresm
noresm_hist_rain = np.load('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_rain_NH.npy')
noresm_hist_lats = np.load('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_lats_NH.npy')
noresm_hist_lons = np.load('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_lons_NH.npy')
noresm_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_sid_NH.csv')
meta_noresm_hist = pd.DataFrame({'sid':noresm_hist_sids.sid ,'centre_lat':noresm_hist_lats[:,4], 'centre_lon':noresm_hist_lons[:,4], 'date':find_dates(noresm_hist_sids,'noleap')})

noresm_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_rain_SH.npy')
noresm_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_lats_SH.npy')
noresm_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_lons_SH.npy')
noresm_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_sid_SH.csv')
meta_noresm_hist_SH = pd.DataFrame({'sid':noresm_hist_sids_SH.sid ,'centre_lat':noresm_hist_lats_SH[:,4], 'centre_lon':noresm_hist_lons_SH[:,4], 'date':find_dates(noresm_hist_sids,'noleap')})

# CMIP6 IPSL
ipsl_hist_rain = np.load('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_rain_NH.npy')
ipsl_hist_lats = np.load('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_lats_NH.npy')
ipsl_hist_lons = np.load('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_lons_NH.npy')
ipsl_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_sid_NH.csv')
meta_ipsl_hist = pd.DataFrame({'sid':ipsl_hist_sids.sid ,'centre_lat':ipsl_hist_lats[:,4], 'centre_lon':ipsl_hist_lons[:,4], 'date':find_dates(ipsl_hist_sids,'gregorian')})

ipsl_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_rain_SH.npy')
ipsl_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_lats_SH.npy')
ipsl_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_lons_SH.npy')
ipsl_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_sid_SH.csv')
meta_ipsl_hist_SH = pd.DataFrame({'sid':ipsl_hist_sids_SH.sid ,'centre_lat':ipsl_hist_lats_SH[:,4], 'centre_lon':ipsl_hist_lons_SH[:,4], 'date':find_dates(ipsl_hist_sids_SH,'gregorian')})

# CMIP6 mri
mri_hist_rain = np.load('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_rain_NH.npy')
mri_hist_lats = np.load('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_lats_NH.npy')
mri_hist_lons = np.load('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_lons_NH.npy')
mri_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_sid_NH.csv')
meta_mri_hist = pd.DataFrame({'sid':mri_hist_sids.sid ,'centre_lat':mri_hist_lats[:,4], 'centre_lon':mri_hist_lons[:,4], 'date':find_dates(mri_hist_sids,'proleptic_gregorian')})

mri_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_rain_SH.npy')
mri_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_lats_SH.npy')
mri_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_lons_SH.npy')
mri_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_sid_SH.csv')
meta_mri_hist_SH = pd.DataFrame({'sid':mri_hist_sids_SH.sid ,'centre_lat':mri_hist_lats_SH[:,4], 'centre_lon':mri_hist_lons_SH[:,4], 'date':find_dates(mri_hist_sids_SH,'proleptic_gregorian')})

# CMIP6 EC-Earth
ecearth_hist_rain = np.load('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_rain_NH.npy')
ecearth_hist_lats = np.load('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_lats_NH.npy')
ecearth_hist_lons = np.load('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_lons_NH.npy')
ecearth_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_sid_NH.csv')
meta_ecearth_hist = pd.DataFrame({'sid':ecearth_hist_sids.sid ,'centre_lat':ecearth_hist_lats[:,4], 'centre_lon':ecearth_hist_lons[:,4], 'date':find_dates(ecearth_hist_sids,'proleptic_gregorian')})

ecearth_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_rain_SH.npy')
ecearth_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_lats_SH.npy')
ecearth_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_lons_SH.npy')
ecearth_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_sid_SH.csv')
meta_ecearth_hist_SH = pd.DataFrame({'sid':ecearth_hist_sids_SH.sid ,'centre_lat':ecearth_hist_lats_SH[:,4], 'centre_lon':ecearth_hist_lons_SH[:,4], 'date':find_dates(ecearth_hist_sids,'proleptic_gregorian')})




ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
print(meta_CMCC_hist_NH)

def calc_nyears(meta):
	nyears = int(meta.year[len(meta.year)-1] - meta.year[0] + 1)
	return nyears




                                           sid  centre_lat  centre_lon  \
0     TC_CMCC-CM2-VHR4_historical_1979_NH_2385        8.25      112.65   
1     TC_CMCC-CM2-VHR4_historical_1979_NH_2388       17.95      120.25   
2     TC_CMCC-CM2-VHR4_historical_1979_NH_2388       19.25      121.05   
3     TC_CMCC-CM2-VHR4_historical_1979_NH_2388       21.25      122.15   
4     TC_CMCC-CM2-VHR4_historical_1979_NH_2388       23.75      123.45   
...                                        ...         ...         ...   
5231  TC_CMCC-CM2-VHR4_historical_2014_NH_5312       14.15      127.75   
5232  TC_CMCC-CM2-VHR4_historical_2014_NH_5312       14.95      128.35   
5233  TC_CMCC-CM2-VHR4_historical_2014_NH_5312       56.15     -157.15   
5234  TC_CMCC-CM2-VHR4_historical_2014_NH_5312       59.35     -155.15   
5235  TC_CMCC-CM2-VHR4_historical_2014_NH_5321       33.75      161.45   

                     date  
0     1979-04-23 00:00:00  
1     1979-06-02 12:00:00  
2     1979-06-02 18:00:00  

In [ ]:
# CMIP6 CMCC_LR
cmcclr_hist_rain = np.load('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_rain_NH.npy')
cmcclr_hist_lats = np.load('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_lats_NH.npy')
cmcclr_hist_lons = np.load('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_lons_NH.npy')
cmcclr_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_sid_NH.csv')
meta_cmcclr_hist = pd.DataFrame({'sid':cmcclr_hist_sids.sid ,'centre_lat':cmcclr_hist_lats[:,4], 'centre_lon':cmcclr_hist_lons[:,4], 'date':find_dates(cmcclr_hist_sids,'365_day')})

cmcclr_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_rain_SH.npy')
cmcclr_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_lats_SH.npy')
cmcclr_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_lons_SH.npy')
cmcclr_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_sid_SH.csv')
meta_cmcclr_hist_SH = pd.DataFrame({'sid':cmcclr_hist_sids_SH.sid ,'centre_lat':cmcclr_hist_lats_SH[:,4], 'centre_lon':cmcclr_hist_lons_SH[:,4], 'date':find_dates(cmcclr_hist_sids_SH,'365_day')})

# CMIP6 NESM3
nesm_hist_rain = np.load('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_rain_NH.npy')
nesm_hist_lats = np.load('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_lats_NH.npy')
nesm_hist_lons = np.load('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_lons_NH.npy')
nesm_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_sid_NH.csv')
meta_nesm_hist = pd.DataFrame({'sid':nesm_hist_sids.sid ,'centre_lat':nesm_hist_lats[:,4], 'centre_lon':nesm_hist_lons[:,4], 'date':find_dates(nesm_hist_sids,'standard')})

nesm_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_rain_SH.npy')
nesm_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_lats_SH.npy')
nesm_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_lons_SH.npy')
nesm_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_sid_SH.csv')
meta_nesm_hist_SH = pd.DataFrame({'sid':nesm_hist_sids_SH.sid ,'centre_lat':nesm_hist_lats_SH[:,4], 'centre_lon':nesm_hist_lons_SH[:,4], 'date':find_dates(nesm_hist_sids_SH,'standard')})

# load trmm
trmm_rain = np.load('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_rain_NH.npy')
trmm_lats = np.load('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_lats_NH.npy')
trmm_lons = np.load('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_lons_NH.npy')
trmm_sids = pd.read_csv('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_sid_NH.csv')
print(imerg_sids.sid.shape)
meta_trmm = pd.DataFrame({'sid':trmm_sids.sid ,'centre_lat':trmm_lats[:,4], 'centre_lon':trmm_lons[:,4], 'hour':trmm_sids.hour, 'year':trmm_sids.year})

# trmm_rain_SH = np.load('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_rain_SH.npy')
# trmm_lats_SH = np.load('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_lats_SH.npy')
# trmm_lons_SH = np.load('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_lons_SH.npy')
# trmm_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_sid_SH.csv')
# print(trmm_sids.sid.shape)
# meta_trmm_SH = pd.DataFrame({'sid':trmm_sids.sid ,'centre_lat':trmm_lats[:,4], 'centre_lon':trmm_lons[:,4], 'hour':trmm_sids.hour})

(8728,)


In [ ]:
print(trmm_rain.shape)

(0, 100, 100)


In [ ]:
# load dataset
# load data
def generate_annual_frequency(meta,rain,units_converter):
	
	# get subset of storms
	dates = meta.date
	dates_unique=dates.drop_duplicates()
	ndates = len(dates_unique)
	tracks_grouped = meta.groupby('date')
	total_rain = np.zeros((ndates,100,100))
	i=0
	drop_idx=[]
	south_idx=[]
	north_idx=[]
	# find time frequency
	for date in dates_unique:
		date_for_rain = tracks_grouped.get_group(date)
		storm_rain = rain[date_for_rain.index,:,:,0]

		# imerg is mm/hr sampled based on ibtracs which is 3 hourly positions
		units_converter_imerg = 3

		# mswep is total rain mm/3hr sampled based on ibtracs which is 3 hourly positions interpolated from 6 hourly
		units_converter_mswep = 1

		# cmip is mm/hr sampled every 6 hourly
		units_converter_cmip = 6


		
		total_daily_rain = np.sum(storm_rain)*units_converter
		
		

		total_rain[i,:,:] = total_daily_rain
		i=i+1

	
	# # get nh and sh datasets
	# composite_storm_rain_nh = composite_storm_rain[north_idx,:,:]
	# composite_storm_rain_sh = composite_storm_rain[south_idx,:,:]
	
	return total_rain,dates




In [ ]:
# Obs and WGAN
total_rain,dates = generate_annual_frequency(meta,real,1)
total_rain_x,dates_x = generate_annual_frequency(meta_og_x,np.expand_dims(real_og_x,axis=-1),1)
ttoal_rain_wgan,dates_wgan = generate_annual_frequency(meta,pred,1)
total_rain_og,dates_og = generate_annual_frequency(meta_og,pred_og,1)
total_rain_og_x,dates_og_x = generate_annual_frequency(meta_og_x,pred_og_x,1)

# imerg
# set imerg=True?
# i don't think we need to multiply rain by 3 hours maybe? because its rain rate over the hour so we assume it rains at the same rate for 3 or 6 hours depending on the track?
# I think the storm track is handling the fact its rain rate and not accumulated rainfall here.
imerg_total_rain,imerg_dates = generate_annual_frequency(meta_imerg,np.expand_dims(imerg_rain,axis=-1),3)
# nyears_imerg = calc_nyears(meta_imerg)

# trmm
# composite_trmm_NH,composite_trmm_SH,nstorms_trmm_nh,nstorms_trmm_sh = generate_annual_frequency(meta_trmm,np.expand_dims(trmm_rain,axis=-1),ibtracks=og_tracks_grouped)
# nyears_trmm = calc_nyears(meta_trmm)

# HighResMIP
total_rain_CMCC_nh,dates_CMCC_nh = generate_annual_frequency(meta_CMCC_hist_NH,np.expand_dims(CMCC_hist_NH_rain,axis=-1),6)
# nyears_CMCC_hist_NH = calc_nyears(meta_CMCC_hist_NH)
total_rain_CMCC_sh,dates_CMCC_sh = generate_annual_frequency(meta_CMCC_hist_SH,np.expand_dims(CMCC_hist_SH_rain,axis=-1),6)
# nyears_CMCC_hist_SH = calc_nyears(meta_CMCC_hist_SH)

total_rain_ecearthhr_nh,dates_exearthhr_nh = generate_annual_frequency(meta_ecearthhr_hist_NH,np.expand_dims(ecearthhr_hist_NH_rain,axis=-1),6)
# nyears_ecearthhr_hist_NH = calc_nyears(meta_ecearthhr_hist_NH)
total_rain_ecearthhr_sh,dates_exearthhr_sh = generate_annual_frequency(meta_ecearthhr_hist_SH,np.expand_dims(ecearthhr_hist_SH_rain,axis=-1),6)
# nyears_ecearthhr_hist_SH = calc_nyears(meta_ecearthhr_hist_SH)

# CMIP6
total_rain_miroc6_nh,dates_miroc6_nh = generate_annual_frequency(meta_miroc6_hist,np.expand_dims(miroc6_hist_rain,axis=-1),6)
# nyears_miroc6_hist_nh = calc_nyears(meta_miroc6_hist)
total_rain_miroc6_nh,dates_miroc6_nh = generate_annual_frequency(meta_miroc6_hist_SH,np.expand_dims(miroc6_hist_rain_SH,axis=-1),6)
# nyears_miroc6_hist_sh = calc_nyears(meta_miroc6_hist_SH)

total_rain_mpi_nh,dates_mpi_nh = generate_annual_frequency(meta_mpi_hist,np.expand_dims(mpi_hist_rain,axis=-1),6,rain_rate=2)
# nyears_mpi_hist_nh = calc_nyears(meta_mpi_hist)
total_rain_mpi_sh,dates_mpi_sh = generate_annual_frequency(meta_mpi_hist_SH,np.expand_dims(mpi_hist_rain_SH,axis=-1),6,rain_rate=2)
# nyears_mpi_hist_sh = calc_nyears(meta_mpi_hist_SH)

total_rain_noresm_nh,dates_noresm_nh = generate_annual_frequency(meta_noresm_hist,np.expand_dims(noresm_hist_rain,axis=-1),6)
# nyears_noresm_hist_nh = calc_nyears(meta_noresm_hist)
total_rain_noresm_nh,dates_noresm_nh = generate_annual_frequency(meta_noresm_hist_SH,np.expand_dims(noresm_hist_rain_SH,axis=-1),6)
# nyears_noresm_hist_sh = calc_nyears(meta_noresm_hist_SH)

total_rain_nesm_nh,dates_nesm_nh = generate_annual_frequency(meta_nesm_hist,np.expand_dims(nesm_hist_rain,axis=-1),6)
# nyears_nesm_hist_nh = calc_nyears(meta_nesm_hist)
total_rain_nesm_sh,dates_nesm_sh = generate_annual_frequency(meta_nesm_hist_SH,np.expand_dims(nesm_hist_rain_SH,axis=-1),6)
# nyears_nesm_hist_sh = calc_nyears(meta_nesm_hist_SH)

total_rain_ipsl_nh,dates_ipsl_nh = generate_annual_frequency(meta_ipsl_hist,np.expand_dims(ipsl_hist_rain,axis=-1),6)
# nyears_ipsl_hist_nh = calc_nyears(meta_ipsl_hist)
total_rain_ipsl_sh,dates_ipsl_sh = generate_annual_frequency(meta_ipsl_hist_SH,np.expand_dims(ipsl_hist_rain_SH,axis=-1),6)
# nyears_ipsl_hist_sh = calc_nyears(meta_ipsl_hist_SH)

total_rain_mri_nh,dates_mri_nh = generate_annual_frequency(meta_mri_hist,np.expand_dims(mri_hist_rain,axis=-1),6)
# nyears_mri_hist_nh = calc_nyears(meta_mri_hist)
total_rain_mri_sh,dates_mri_sh  = generate_annual_frequency(meta_mri_hist_SH,np.expand_dims(mri_hist_rain_SH,axis=-1),6)
# nyears_mri_hist_sh = calc_nyears(meta_mri_hist_SH)

total_rain_ecearth_nh,dates_ecearth_nh = generate_annual_frequency(meta_ecearth_hist,np.expand_dims(ecearth_hist_rain,axis=-1),6)
# nyears_ecearth_hist_nh = calc_nyears(meta_ecearth_hist)
total_rain_ecearth_sh,dates_ecearth_sh = generate_annual_frequency(meta_ecearth_hist_SH,np.expand_dims(ecearth_hist_rain_SH,axis=-1),6)
# nyears_ecearth_hist_sh = calc_nyears(meta_ecearth_hist_SH)

total_rain_cmcclr_nh,dates_cmcclr_nh = generate_annual_frequency(meta_cmcclr_hist,np.expand_dims(cmcclr_hist_rain,axis=-1),6)
# nyears_cmcclr_hist_nh = calc_nyears(meta_cmcclr_hist)
total_rain_cmcclr_sh,dates_cmcclr_sh = generate_annual_frequency(meta_cmcclr_hist_SH,np.expand_dims(cmcclr_hist_rain_SH,axis=-1),6)
# nyears_cmcclr_hist_sh = calc_nyears(meta_cmcclr_hist_SH)

AttributeError: 'DataFrame' object has no attribute 'date'

In [ ]:
# calculate number of storms for sh / nh weight ratio for final plot
nstorms_cmip6_hist_nh = nstorms_miroc6_hist_nh + nstorms_mpi_hist_nh + nstorms_noresm_hist_nh + nstorms_nesm_hist_nh + nstorms_ipsl_hist_nh + nstorms_mri_hist_nh + nstorms_ecearth_hist_nh + nstorms_cmcclr_hist_nh
nstorms_cmip6_hist_sh = nstorms_miroc6_hist_sh + nstorms_mpi_hist_sh + nstorms_noresm_hist_sh + nstorms_nesm_hist_sh + nstorms_ipsl_hist_sh + nstorms_mri_hist_sh + nstorms_ecearth_hist_sh + nstorms_cmcclr_hist_sh

nstorms_hrmip_hist_nh = nstorms_CMCC_hist_nh + nstorms_ecearthhr_hist_nh
nstorms_hrmip_hist_sh = nstorms_CMCC_hist_sh + nstorms_ecearthhr_hist_sh

print(nstorms_cmip6_hist_nh)
print(nstorms_cmip6_hist_sh)
print(nstorms_cmip6_hist_sh + nstorms_cmip6_hist_nh)
print(nstorms_cmip6_hist_nh / (nstorms_cmip6_hist_sh + nstorms_cmip6_hist_nh))
print(nstorms_cmip6_hist_sh / (nstorms_cmip6_hist_sh + nstorms_cmip6_hist_nh))

# nstorms_cmip6_ssp585_nh = nstorms_miroc6_ssp585_nh + nstorms_mpi_ssp585_nh + nstorms_noresm_ssp585_nh + nstorms_nesm_ssp585_nh + nstorms_ipsl_ssp585_nh + nstorms_mri_ssp585_nh + nstorms_ecearth_ssp585_nh + nstorms_cmcclr_ssp585_nh
# nstorms_cmip6_ssp585_sh = nstorms_miroc6_ssp585_sh + nstorms_mpi_ssp585_sh + nstorms_noresm_ssp585_sh + nstorms_nesm_ssp585_sh + nstorms_ipsl_ssp585_sh + nstorms_mri_ssp585_sh + nstorms_ecearth_ssp585_sh + nstorms_cmcclr_ssp585_sh

In [ ]:
# define cmap
precip_cmap,precip_norm = make_cmap()

In [ ]:

fig,axes = plt.subplots(10,5,figsize=(20.5, 50),gridspec_kw={'width_ratios': [1, 1, 1, 1, 0.05]})

# MSWEP Obs
im_1 = axes[0,0].contourf(composite_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[0,0].annotate('Max: %.1f' % np.max(composite_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
axes[0,0].annotate('Storms: %.0f' % nstorms_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
# axes[0,0].annotate('Storms per year: %.0f' % (int(nstorms_nh) / nyea), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
im_2 = axes[0,1].contourf(composite_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[0,1].annotate('Max: %.1f' % np.max(composite_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
axes[0,1].annotate('Storms: %.0f' % nstorms_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
im_3 = axes[0,2].contourf(composite_nh_x,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[0,2].annotate('Max: %.1f' % np.max(composite_nh_x), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
axes[0,2].annotate('Storms: %.0f' % nstorms_x_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
im_4 = axes[0,3].contourf(composite_sh_x,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[0,3].annotate('Max: %.1f' % np.max(composite_sh_x), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
axes[0,3].annotate('Storms: %.0f' % nstorms_x_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

# # IMERG Obs
im_5 = axes[1,0].contourf(composite_imerg_NH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[1,0].annotate('Max: %.1f' % np.max(composite_imerg_NH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
im_6 = axes[1,1].contourf(composite_imerg_SH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[1,1].annotate('Max: %.1f' % np.max(composite_imerg_SH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')

# im_5b = axes[1,2].contourf(imerg_NH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[1,2].annotate('Max: %.1f' % np.max(imerg_NH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# im_6b = axes[1,3].contourf(imerg_SH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[1,3].annotate('Max: %.1f' % np.max(imerg_SH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')

# # TRMM Obs
im_5 = axes[1,2].contourf(composite_trmm_NH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[1,2].annotate('Max: %.1f' % np.max(composite_trmm_NH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
im_6 = axes[1,3].contourf(composite_trmm_SH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[1,3].annotate('Max: %.1f' % np.max(composite_trmm_SH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')

# im_5b = axes[1,2].contourf(imerg_NH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[1,2].annotate('Max: %.1f' % np.max(imerg_NH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# im_6b = axes[1,3].contourf(imerg_SH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[1,3].annotate('Max: %.1f' % np.max(imerg_SH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')

# HighresMIP
im_5 = axes[3,0].contourf(composite_CMCC_hist_NH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[3,0].annotate('Max: %.1f' % np.max(composite_CMCC_hist_NH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[3,0].annotate('Storms: %.0f' % nstorms_CMCC_hist_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[3,0].annotate('Storms per year: %.0f' % (int(nstorms_CMCC_hist_nh) / nyears_CMCC_hist_NH), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
im_6 = axes[3,1].contourf(composite_CMCC_hist_SH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[3,1].annotate('Max: %.1f' % np.max(composite_CMCC_hist_SH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[3,1].annotate('Storms: %.0f' % nstorms_CMCC_hist_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[3,1].annotate('Storms per year: %.0f' % (int(nstorms_CMCC_hist_sh) / nyears_CMCC_hist_SH) , xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

im_5b = axes[3,2].contourf(composite_ecearthhr_hist_NH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[3,2].annotate('Max: %.1f' % np.max(composite_ecearthhr_hist_NH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[3,2].annotate('Storms: %.0f' % nstorms_ecearthhr_hist_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[3,2].annotate('Storms per year: %.0f' % (int(nstorms_ecearthhr_hist_nh) / nyears_ecearthhr_hist_NH), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
im_6b = axes[3,3].contourf(composite_ecearthhr_hist_SH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[3,3].annotate('Max: %.1f' % np.max(composite_ecearthhr_hist_SH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[3,3].annotate('Storms: %.0f' % nstorms_ecearthhr_hist_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[3,3].annotate('Storms: %.0f' % (int(nstorms_ecearthhr_hist_sh) / nyears_ecearthhr_hist_SH), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

# CMIP6 hist
im_7 = axes[4,0].contourf(composite_miroc6_hist_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[4,0].annotate('Max: %.1f' % np.max(composite_miroc6_hist_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[4,0].annotate('Storms: %.0f' % nstorms_miroc6_hist_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[4,0].annotate('Storms per year: %.0f' % (int(nstorms_miroc6_hist_nh) / nyears_miroc6_hist_nh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
im_8 = axes[4,1].contourf(composite_miroc6_hist_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[4,1].annotate('Max: %.1f' % np.max(composite_miroc6_hist_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[4,1].annotate('Storms: %.0f' % nstorms_miroc6_hist_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[4,1].annotate('Storms per year: %.0f' % (int(nstorms_miroc6_hist_sh) / nyears_miroc6_hist_sh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

im_7b = axes[4,2].contourf(composite_mpi_hist_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[4,2].annotate('Max: %.1f' % np.max(composite_mpi_hist_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[4,2].annotate('Storms: %.0f' % nstorms_mpi_hist_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[4,2].annotate('Storm per year: %.0f' % (int(nstorms_mpi_hist_nh) / nyears_mpi_hist_nh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
im_8b = axes[4,3].contourf(composite_mpi_hist_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[4,3].annotate('Max: %.1f' % np.max(composite_mpi_hist_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[4,3].annotate('Storms: %.0f' % nstorms_mpi_hist_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[4,3].annotate('Storms per year: %.0f' % (int(nstorms_mpi_hist_sh) / nyears_mpi_hist_sh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

im_7c = axes[5,0].contourf(composite_noresm_hist_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[5,0].annotate('Max: %.1f' % np.max(composite_noresm_hist_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[5,0].annotate('Storms: %.0f' % nstorms_noresm_hist_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[5,0].annotate('Storms per year: %.0f' % (int(nstorms_noresm_hist_nh) / nyears_noresm_hist_nh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
im_8c = axes[5,1].contourf(composite_noresm_hist_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[5,1].annotate('Max: %.1f' % np.max(composite_noresm_hist_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[5,1].annotate('Storms: %.0f' % nstorms_noresm_hist_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[5,1].annotate('Storms per year: %.0f' % (int(nstorms_noresm_hist_sh) / nyears_noresm_hist_sh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

im_7d = axes[5,2].contourf(composite_nesm_hist_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[5,2].annotate('Max: %.1f' % np.max(composite_nesm_hist_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[5,2].annotate('Storms: %.0f' % nstorms_nesm_hist_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[5,2].annotate('Storms per year: %.0f' % (int(nstorms_nesm_hist_nh) / nyears_nesm_hist_nh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
im_8c = axes[5,3].contourf(composite_nesm_hist_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[5,3].annotate('Max: %.1f' % np.max(composite_nesm_hist_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[5,3].annotate('Storms: %.0f' % nstorms_nesm_hist_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[5,3].annotate('Storms per year: %.0f' % (int(nstorms_nesm_hist_sh) / nyears_nesm_hist_sh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

axes[6,0].contourf(composite_ipsl_hist_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[6,0].annotate('Max: %.1f' % np.max(composite_ipsl_hist_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[6,0].annotate('Storms: %.0f' % nstorms_ipsl_hist_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[6,0].annotate('Storms per year: %.0f' % (int(nstorms_ipsl_hist_nh) / nyears_ipsl_hist_nh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[6,1].contourf(composite_ipsl_hist_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[6,1].annotate('Max: %.1f' % np.max(composite_ipsl_hist_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[6,1].annotate('Storms: %.0f' % nstorms_ipsl_hist_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[6,1].annotate('Storms: %.0f' % (int(nstorms_ipsl_hist_sh) / nyears_ipsl_hist_sh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

axes[6,2].contourf(composite_mri_hist_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[6,2].annotate('Max: %.1f' % np.max(composite_mri_hist_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[6,2].annotate('Storms: %.0f' % nstorms_mri_hist_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[6,2].annotate('Storms per year: %.0f' % (int(nstorms_mri_hist_nh) / nyears_mri_hist_nh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[6,3].contourf(composite_mri_hist_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[6,3].annotate('Max: %.1f' % np.max(composite_mri_hist_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[6,3].annotate('Storms: %.0f' % nstorms_mri_hist_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[6,3].annotate('Storms per year: %.0f' % (int(nstorms_mri_hist_sh) / nyears_mri_hist_sh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

axes[7,0].contourf(composite_ecearth_hist_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[7,0].annotate('Max: %.1f' % np.max(composite_ecearth_hist_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[7,0].annotate('Storms: %.0f' % nstorms_ecearth_hist_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[7,0].annotate('Storms per year: %.0f' % (int(nstorms_ecearth_hist_nh) / nyears_ecearth_hist_nh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[7,1].contourf(composite_ecearth_hist_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[7,1].annotate('Max: %.1f' % np.max(composite_ecearth_hist_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[7,1].annotate('Storms: %.0f' % nstorms_ecearth_hist_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[7,1].annotate('Storms per year: %.0f' % (int(nstorms_ecearth_hist_sh) / nyears_ecearth_hist_sh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

axes[7,2].contourf(composite_cmcclr_hist_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[7,2].annotate('Max: %.1f' % np.max(composite_cmcclr_hist_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[7,2].annotate('Storms: %.0f' % nstorms_cmcclr_hist_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[7,2].annotate('Storms per year: %.0f' % (int(nstorms_cmcclr_hist_nh) / nyears_cmcclr_hist_nh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[7,3].contourf(composite_cmcclr_hist_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[7,3].annotate('Max: %.1f' % np.max(composite_cmcclr_hist_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[7,3].annotate('Storms: %.0f' % nstorms_cmcclr_hist_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
axes[7,3].annotate('Storms per year: %.0f' % (int(nstorms_cmcclr_hist_sh) / nyears_cmcclr_hist_sh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')


# OG WGAN
im_og_nh = axes[8,0].contourf(composite_og_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[8,0].annotate('Max: %.1f' % np.max(composite_og_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
im_og_sh = axes[8,1].contourf(composite_og_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[8,1].annotate('Max: %.1f' % np.max(composite_og_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
im_og_x_nh = axes[8,2].contourf(composite_og_x_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[8,2].annotate('Max: %.1f' % np.max(composite_og_x_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
im_og_x_sh = axes[8,3].contourf(composite_og_x_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[8,3].annotate('Max: %.1f' % np.max(composite_og_x_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')

# Scalar WGAN
im_pred_nh = axes[9,0].contourf(composite_wgan_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[9,0].annotate('Max: %.1f' % np.max(composite_wgan_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
im_pred_sh = axes[9,1].contourf(composite_wgan_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[9,1].annotate('Max: %.1f' % np.max(composite_wgan_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')

# Set the tick positions
t_p = [0,10,20,30,40,50,60,70,80,90,100]
t_l = [-5,-4,-3,-2,-1,0,1,2,3,4,5]
for i in range(10):
	for j in range(4):
		axes[i,j].set_xticks(t_p)
		axes[i,j].set_yticks(t_p)
		# Set the tick labels
		axes[i,j].set_xticklabels(t_l)
		axes[i,j].set_yticklabels(t_l)


axes[0,0].set_title('MSWEP (NH)')
axes[0,0].set_xlabel('Longitude (degrees)')
axes[0,0].set_ylabel('Latitude (degrees)')
axes[0,1].set_title('MSWEP (SH)')
# axes[0,1].set_xlabel('Longitude (degrees)')
# add **font to change font

axes[0,2].set_title('MSWEP extreme 100 (NH)')
axes[0,3].set_title('MSWEP extreme 100 (SH)')

axes[1,0].set_title('IMERG (NH)')
axes[1,1].set_title('IMERG (SH)')
axes[1,2].set_title('IMERG extreme 100 (NH)')
axes[1,3].set_title('IMERG extreme 100 (SH)')

axes[2,0].set_title('TRMM (NH)')
axes[2,1].set_title('TRMM (SH)')
axes[2,2].set_title('TRMM extreme 100 (NH)')
axes[2,3].set_title('TRMM extreme 100 (SH)')

axes[3,0].set_title('HighResMIP CMCC Hist (NH)')
axes[3,1].set_title('HighResMIP CMCC Hist (SH)')
axes[3,2].set_title('HighResMIP EC-Earth-3P-HR Hist (NH)')
axes[3,3].set_title('HighResMIP EC-Earth-3P-HR Hist (SH)')

axes[4,0].set_title('CMIP6 MIROC6 Hist (NH)')
axes[4,1].set_title('CMIP6 MIROC6 Hist (SH)')
axes[4,2].set_title('CMIP6 MPI-ESM1-2-LR (NH)')
axes[4,3].set_title('CMIP6 MPI-ESM1-2-LR (SH)')

axes[5,0].set_title('CMIP6 NORESM hist (NH)')
axes[5,1].set_title('CMIP6 NORESM hist (SH)')
axes[5,2].set_title('CMIP6 NESM3 (NH)')
axes[5,3].set_title('CMIP6 NESM3 (SH)')

axes[6,0].set_title('CMIP6 IPSL-CM6A-LR (NH)')
axes[6,1].set_title('CMIP6 IPSL-CM6A-LR (SH)')
axes[6,2].set_title('CMIP6 MRI-ESM2-0 (NH)')
axes[6,3].set_title('CMIP6 MRI-ESM2-0 (SH)')

axes[7,0].set_title('CMIP6 EC-Earth3 (NH)')
axes[7,1].set_title('CMIP6 EC-Earth3 (SH)')
axes[7,2].set_title('CMIP6 CMCC-ESM2 (NH)')
axes[7,3].set_title('CMIP6 CMCC-ESM2 (SH)')

axes[8,0].set_title('2D WGAN (NH)')
axes[8,1].set_title('2D WGAN (SH)')
axes[8,2].set_title('2D WGAN extreme (NH)')
axes[8,3].set_title('2D WGAN extreme (SH)')

axes[9,0].set_title('Scalar WGAN applied to ERA5 (NH)')
axes[9,1].set_title('Scalar WGAN applied to ERA5 (SH)')
axes[9,2].set_title('Scalar WGAN extreme applied to ERA5 (NH)')
axes[9,3].set_title('Scalar WGAN extreme applied to ERA5 (SH)')




plt.colorbar(im_2,ticks=[0,20,40,60,80,100,120,140],cax=axes[0,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[1,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[2,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[3,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[4,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[5,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[6,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[7,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[8,4])


plt.savefig('composite_rain5.png',bbox_inches='tight')

In [ ]:
# save everything
fig_dir = '/user/home/al18709/work/figures/figure_1/'



# HighresMIP
np.save(fig_dir + 'composite_CMCC_hist_NH.npy',composite_CMCC_hist_NH)
np.save(fig_dir + 'composite_CMCC_hist_SH.npy', composite_CMCC_hist_SH)

np.save(fig_dir + 'composite_ecearthhr_hist_NH.npy',composite_ecearthhr_hist_NH)
np.save(fig_dir + 'composite_ecearthhr_hist_SH.npy',composite_ecearthhr_hist_SH)

# CMIP6 hist
np.save(fig_dir + 'composite_miroc6_hist_nh.npy',composite_miroc6_hist_nh)
np.save(fig_dir + 'composite_miroc6_hist_sh.npy',composite_miroc6_hist_sh)

np.save(fig_dir + 'composite_mpi_hist_nh.npy',composite_mpi_hist_nh)
np.save(fig_dir + 'composite_mpi_hist_sh.npy',composite_mpi_hist_sh)

np.save(fig_dir + 'composite_noresm_hist_nh.npy',composite_noresm_hist_nh)
np.save(fig_dir + 'composite_noresm_hist_sh.npy',composite_noresm_hist_sh)

# np.save(fig_dir + composite_taiesm_hist_nh,'composite_taiesm_hist_nh.npy')
# np.save(fig_dir + composite_taiesm_hist_sh,'composite_taiesm_hist_sh.npy')

np.save(fig_dir + 'composite_ipsl_hist_nh.npy',composite_ipsl_hist_nh)
np.save(fig_dir + 'composite_ipsl_hist_sh.npy',composite_ipsl_hist_sh)

np.save(fig_dir + 'composite_mri_hist_nh.npy',composite_mri_hist_nh)
np.save(fig_dir + 'composite_mri_hist_sh.npy',composite_mri_hist_sh)

np.save(fig_dir + 'composite_ecearth_hist_nh.npy',composite_ecearth_hist_nh)
np.save(fig_dir + 'composite_ecearth_hist_sh.npy',composite_ecearth_hist_sh)

np.save(fig_dir + 'composite_cmcclr_hist_nh.npy',composite_cmcclr_hist_nh)
np.save(fig_dir + 'composite_cmcclr_hist_sh.npy',composite_cmcclr_hist_sh)

# np.save(fig_dir + composite_bcc_hist_nh,'composite_bcc_hist_nh.npy')
# np.save(fig_dir + composite_bcc_hist_sh,'composite_bcc_hist_sh.npy')

np.save(fig_dir + 'composite_nems_hist_nh.npy',composite_nesm_hist_nh)
np.save(fig_dir + 'composite_nems_hist_sh.npy',composite_nesm_hist_sh)


# OG WGAN
# np.save(composite_og_nh,'composite_og_nh.npy')
# np.save(composite_og_sh,'composite_og_sh.npy')
# np.save(composite_og_x_nh,'composite_og_x_nh.npy')
# np.save(composite_og_x_nh,'composite_og_x_nh.npy')
# np.save(composite_og_x_sh,'composite_og_x_sh.npy')

# # Scalar WGAN
# np.save(composite_wgan_nh,'composite_wgan_nh.npy')
# np.save(composite_wgan_sh,'composite_wgan_sh.npy')
